In [1]:
import torch
import torch.nn as nn

class PatchPartition(nn.Module):
    def __init__(self, patch_size=4):
        super(PatchPartition, self).__init__()
        self.patch_size = patch_size

    def forward(self, x):
        B, H, W, C = x.shape
        x = x.view(B,
                   H // self.patch_size, self.patch_size,
                   W // self.patch_size, self.patch_size,
                   C)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()  # (B, H//p, W//p, p, p, C)
        x = x.view(B, H // self.patch_size * W // self.patch_size, -1)  # (B, H//p, W//p, patch_dim)
        return x

In [2]:
input_tensor = torch.randn(2, 224, 224, 3)  # (B, H, W, C)
patch_partition = PatchPartition(patch_size=4)
output = patch_partition(input_tensor)
print("Output shape:", output.shape)

Output shape: torch.Size([2, 3136, 48])


In [3]:
class LinearEmbedding(nn.Module):
    def __init__(self, input_dim=48, embed_dim=96):
        super(LinearEmbedding, self).__init__()
        self.proj = nn.Linear(input_dim, embed_dim)

    def forward(self, x):
        # x shape: (B, H//patch_size, W//patch_size, patch_dim)
        B, L , _ = x.shape
        x = self.proj(x)  
        return x  

Linear embedding uses weight matrix of shape 96 x 48 to project the 48 sized input to 96

Y=WX+b

In [4]:
linear_embedding = LinearEmbedding(input_dim=48, embed_dim=96)
output = linear_embedding(output)
print(output.shape)

torch.Size([2, 3136, 96])


In [5]:
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size,
               W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5)
    windows = windows.contiguous().view(-1, window_size , window_size, C)
    return windows

In [6]:
def window_reverse(windows, window_size, H, W):
    B = int(windows.shape[0] / (H // window_size * W // window_size))
    x = windows.view(B,
                     H // window_size,
                     W // window_size,
                     window_size,
                     window_size,
                     -1)
    x = x.permute(0, 1, 3, 2, 4, 5)
    x = x.contiguous().view(B, H, W, -1)
    return x

In [7]:
class WindowAttention(nn.Module):
    def __init__(self, dim, window_size, num_heads):
        super().__init__()
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        num_rel = (2 * window_size - 1) ** 2
        self.relative_bias = nn.Parameter(torch.zeros(num_rel, num_heads))
        coords = torch.stack(torch.meshgrid(
            torch.arange(window_size), torch.arange(window_size), indexing='ij'
        ))  # 2, Wh, Ww
        coords = coords.flatten(1)  # 2, Wh*Ww
        rel = coords[:, :, None] - coords[:, None, :]  # 2, N, N
        rel = rel.permute(1, 2, 0).contiguous()  # N, N, 2
        rel[:, :, 0] += window_size - 1
        rel[:, :, 1] += window_size - 1
        rel[:, :, 0] *= 2 * window_size - 1
        index = rel.sum(-1)
        self.register_buffer('relative_index', index)

        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.proj = nn.Linear(dim, dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):  # x: (nW*B, N, C)
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv.permute(2, 0, 3, 1, 4)
        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))  # (B_, heads, N, N)

        bias = self.relative_bias[self.relative_index.view(-1)]
        bias = bias.view(N, N, -1).permute(2, 0, 1)
        attn = attn + bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N)
            attn = attn + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)

        attn = self.softmax(attn)
        out = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        out = self.proj(out)
        return out

In [8]:
class Mlp(nn.Module):
    def __init__(self, dim, mlp_ratio=4, output_dim=None):
        super().__init__()
        hidden = int(dim * mlp_ratio)
        out_dim = output_dim if output_dim is not None else dim
        self.fc1 = nn.Linear(dim, hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden, out_dim)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))

In [9]:
class SwinBlock(nn.Module):
    def __init__(self, dim, input_resolution, num_heads,
                 window_size=7, shift_size=0, mlp_ratio=4, output_dim=None):
        super().__init__()
        H, W = input_resolution
        assert 0 <= shift_size < window_size, "shift_size must be in [0, window_size)"
        self.dim = dim
        self.H, self.W = H, W
        self.window_size = window_size
        self.shift_size = shift_size

        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(dim, window_size, num_heads)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = Mlp(dim, mlp_ratio, output_dim)

        if self.shift_size > 0:
            img_mask = torch.zeros((1, H, W, 1))
            h_slices = (slice(0, -window_size), slice(-window_size, -shift_size), slice(-shift_size, None))
            w_slices = (slice(0, -window_size), slice(-window_size, -shift_size), slice(-shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, window_size)
            mask_windows = mask_windows.view(-1, window_size * window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):  # x: (B, H*W, C)
        B, L, C = x.shape
        shortcut = x
        x = self.norm1(x).view(B, self.H, self.W, C)

        # cyclic shift
        if self.shift_size > 0:
            x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))

        # partition windows
        x_windows = window_partition(x, self.window_size)  
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C) # (nW*B, M*M, C)
        # attention
        attn_windows = self.attn(x_windows, mask=self.attn_mask)
        # merge windows
        x = window_reverse(attn_windows, self.window_size, self.H, self.W)

        # reverse shift
        if self.shift_size > 0:
            x = torch.roll(x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))

        x = x.view(B, L, C)
        # residual + MLP
        x = shortcut + x
        x = x + self.mlp(self.norm2(x))
        return x

In [10]:
class PatchMerging(nn.Module):
    def __init__(self, input_resolution, dim):
        super().__init__()
        H, W = input_resolution
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = nn.LayerNorm(4 * dim)

    def forward(self, x):  # x: (B, H*W, C)
        B, L, C = x.shape
        H, W = self.input_resolution
        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = torch.cat([x0, x1, x2, x3], -1)  # (B, H/2, W/2, 4*C)
        x = x.view(B, -1, 4 * C)
        x = self.norm(x)
        x = self.reduction(x)
        return x

In [11]:
class SwinTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=4, in_chans=3,
                 embed_dim=96, depths=[2,2,6,2], num_heads=[3,6,12,24], window_size=7,
                 num_classes=1000):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        # 1) Patch partition + embedding
        self.partition = PatchPartition(patch_size)
        patch_dim = patch_size*patch_size*in_chans
        self.embedding = LinearEmbedding(patch_dim, embed_dim)
        # compute resolution after patch
        H = W = img_size // patch_size
        # build stages
        self.stages = nn.ModuleList()
        dims = [embed_dim, embed_dim*2, embed_dim*4, embed_dim*8]
        for i in range(4):
            blocks = []
            for j in range(depths[i]):
                shift = 0 if (j % 2 == 0) else window_size//2
                blocks.append(
                    SwinBlock(dims[i], (H,W), num_heads[i], window_size, shift)
                )
            stage = nn.Sequential(*blocks)
            self.stages.append(stage)
            if i < 3:
                # downsample
                self.stages.append(PatchMerging((H,W), dims[i]))
                H //= 2; W //= 2
        # final norm
        self.norm = nn.LayerNorm(dims[-1])
                # classification head
        self.head = nn.Linear(dims[-1], num_classes)

    def forward(self, x):  # x: (B, C, H, W)
        B,C,H,W = x.shape
        x = x.permute(0,2,3,1).contiguous()  # to (B, H, W, C)
        x = self.partition(x)              # (B, H/p*W/p, patch_dim)
        x = self.embedding(x)              # (B, N, embed_dim)
        for layer in self.stages:
            x = layer(x)
        x = self.norm(x)
        x = x.mean(1)  # global average
        return self.head(x)  # (B, num_classes)

In [12]:
import os
import random
import csv
import gc
from glob import glob
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# Set seeds for reproducibility
random.seed(42)
torch.manual_seed(42)

# === CONFIG ===
ROOT_DIR = "/kaggle/input/imagenet1k0"
NUM_CLASSES = 10
TRAIN_PER_CLASS = 500
VAL_PER_CLASS = 50
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LOG_FILE = "log.csv"
SAVE_PATH = "best_model.pth"

# === Transforms ===
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# === Dataset Class ===
class ImageDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# === Sample 10 classes ===
all_classes = sorted([d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))])
selected_classes = all_classes[:NUM_CLASSES]
print(selected_classes)
# === Collect images per class ===
class_to_idx = {cls_name: i for i, cls_name in enumerate(selected_classes)}
train_samples, val_samples = [], []

for cls in selected_classes:
    cls_path = os.path.join(ROOT_DIR, cls)
    images = sorted(glob(os.path.join(cls_path, "*.*")))
    selected_images = random.sample(images, TRAIN_PER_CLASS + VAL_PER_CLASS)
    train_images = selected_images[:TRAIN_PER_CLASS]
    val_images = selected_images[TRAIN_PER_CLASS:]
    train_samples.extend([(img, class_to_idx[cls]) for img in train_images])
    val_samples.extend([(img, class_to_idx[cls]) for img in val_images])

# === DataLoaders ===
train_loader = DataLoader(ImageDataset(train_samples, transform), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ImageDataset(val_samples, transform), batch_size=BATCH_SIZE)

# === Model, Loss, Optimizer ===
model = SwinTransformer(num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# === CSV Logger ===
with open(LOG_FILE, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "Epoch", "Train Loss", "Train Acc", "Train Precision", "Train Recall", "Train F1",
        "Val Acc", "Val Precision", "Val Recall", "Val F1"
    ])

# === Training Loop ===
best_f1 = 0

for epoch in range(EPOCHS):
    model.train()
    train_preds, train_labels = [], []
    train_loss = 0

    for inputs, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = outputs.argmax(dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = (torch.tensor(train_preds) == torch.tensor(train_labels)).float().mean().item()
    train_precision = precision_score(train_labels, train_preds, average='macro',zero_division=0)
    train_recall = recall_score(train_labels, train_preds, average='macro',zero_division=0)
    train_f1 = f1_score(train_labels, train_preds, average='macro',zero_division=0)
    print(f"Epoch {epoch+1} Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | P: {train_precision:.4f} | R: {train_recall:.4f} | F1: {train_f1:.4f}")

    # === Validation ===
    model.eval()
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}] Validation"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = (torch.tensor(val_preds) == torch.tensor(val_labels)).float().mean().item()
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)
    print(f"Epoch {epoch+1} Val Acc: {val_acc:.4f} | P: {val_precision:.4f} | R: {val_recall:.4f} | F1: {val_f1:.4f}")

    # === Save best model ===
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), SAVE_PATH)
        print("✅ Best model updated and saved!")

    # === Log to CSV ===
    with open(LOG_FILE, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            epoch + 1,
            round(train_loss, 4),
            round(train_acc, 4),
            round(train_precision, 4),
            round(train_recall, 4),
            round(train_f1, 4),
            round(val_acc, 4),
            round(val_precision, 4),
            round(val_recall, 4),
            round(val_f1, 4)
        ])

    # Optional: Free up memory
    torch.cuda.empty_cache()
    gc.collect()

['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009']


[Epoch 1] Training: 100%|██████████| 79/79 [01:38<00:00,  1.25s/it]


Epoch 1 Train Loss: 163.3260 | Acc: 0.2174 | P: 0.2160 | R: 0.2174 | F1: 0.2157


[Epoch 1] Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]


Epoch 1 Val Acc: 0.2580 | P: 0.2096 | R: 0.2580 | F1: 0.1768
✅ Best model updated and saved!


...

Epoch 99 Val Acc: 0.5980 | P: 0.5963 | R: 0.5980 | F1: 0.5953


[Epoch 100] Training: 100%|██████████| 79/79 [00:57<00:00,  1.38it/s]


Epoch 100 Train Loss: 0.3587 | Acc: 0.9980 | P: 0.9980 | R: 0.9980 | F1: 0.9980


[Epoch 100] Validation: 100%|██████████| 8/8 [00:03<00:00,  2.41it/s]


Epoch 100 Val Acc: 0.6180 | P: 0.6132 | R: 0.6180 | F1: 0.6148
